# Django та REST. Огляд Django REST Framework (DRF)

## Мета лекції

Після цієї лекції ви:

* зрозумієте, що таке REST та API;
* знатимете різницю між Django (templates) та Django REST Framework;
* навчитеся створювати просте API;
* зрозумієте ролі Serializer, View, URL у DRF;
* знатимете базові принципи побудови RESTful сервісів.

---

## 1. Що таке REST і API

### API (Application Programming Interface)

API — це інтерфейс для взаємодії між програмами.

Приклад:

* Frontend (React, mobile app)
* Backend (Django)
* Обмін даними через **JSON**

---

### REST (Representational State Transfer)

REST — це архітектурний стиль побудови API.

Основні принципи REST:

* клієнт–сервер
* stateless (сервер не зберігає стан клієнта)
* ресурси (resources)
* HTTP-методи

---

## 2. HTTP-методи у REST

| Метод  | Призначення      |
| ------ | ---------------- |
| GET    | Отримати дані    |
| POST   | Створити ресурс  |
| PUT    | Повністю оновити |
| PATCH  | Частково оновити |
| DELETE | Видалити         |

Приклад REST-URL:

```
GET /api/posts/
GET /api/posts/1/
POST /api/posts/
```

---

## 3. Django vs Django REST Framework

### Django (Templates)

* HTML-відповідь
* server-side rendering
* браузер як клієнт

### DRF

* JSON-відповідь
* frontend і backend розділені
* mobile apps, SPA, microservices

📌 **DRF = стандарт для API у Django**

---

## 4. Встановлення Django REST Framework

```bash
pip install djangorestframework
```

**settings.py**

```python
INSTALLED_APPS = [
    ...
    'rest_framework',
]
```

---

## 5. Перше API без моделей

### views.py

```python
from rest_framework.decorators import api_view
from rest_framework.response import Response

@api_view(['GET'])
def hello_api(request):
    return Response({'message': 'Hello API'})
```

### urls.py

```python
from django.urls import path
from .views import hello_api

urlpatterns = [
    path('hello/', hello_api),
]
```

📌 DRF автоматично повертає JSON

---

## 6. Response vs JsonResponse

❌ Django way:

```python
from django.http import JsonResponse
return JsonResponse({'msg': 'hi'})
```

✅ DRF way:

```python
from rest_framework.response import Response
return Response({'msg': 'hi'})
```

---

## 7. Модель як ресурс

### models.py

```python
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()

    def __str__(self):
        return self.title
```

---

## 8. Serializer — серце DRF

Serializer:

* перетворює Model ↔ JSON
* виконує валідацію

### serializers.py

```python
from rest_framework import serializers
from .models import Post

class PostSerializer(serializers.ModelSerializer):
    class Meta:
        model = Post
        fields = ['id', 'title', 'content']
```

---

## 9. API View з моделлю

```python
from rest_framework.decorators import api_view
from rest_framework.response import Response
from .models import Post
from .serializers import PostSerializer

@api_view(['GET'])
def post_list(request):
    posts = Post.objects.all()
    serializer = PostSerializer(posts, many=True)
    return Response(serializer.data)
```

---

## 10. POST-запит у DRF

```python
@api_view(['POST'])
def post_create(request):
    serializer = PostSerializer(data=request.data)
    if serializer.is_valid():
        serializer.save()
        return Response(serializer.data, status=201)
    return Response(serializer.errors, status=400)
```

📌 `request.data` — замість `request.POST`

---

## 11. Browsable API

DRF надає **інтерактивний веб-інтерфейс**:

* перегляд даних
* відправка POST
* тестування API

📌 Дуже зручно для QA та розробників

---

## 12. Class-Based Views (коротко)

```python
from rest_framework.views import APIView

class PostListView(APIView):
    def get(self, request):
        ...
```

Або ще простіше:

```python
from rest_framework.generics import ListCreateAPIView

class PostListView(ListCreateAPIView):
    queryset = Post.objects.all()
    serializer_class = PostSerializer
```

---

## 13. ViewSets та Routers (огляд)

```python
from rest_framework.viewsets import ModelViewSet

class PostViewSet(ModelViewSet):
    queryset = Post.objects.all()
    serializer_class = PostSerializer
```

```python
from rest_framework.routers import DefaultRouter

router = DefaultRouter()
router.register('posts', PostViewSet)
```

📌 CRUD без boilerplate

---

## 14. Статуси HTTP

| Код | Значення     |
| --- | ------------ |
| 200 | OK           |
| 201 | Created      |
| 400 | Bad Request  |
| 401 | Unauthorized |
| 403 | Forbidden    |
| 404 | Not Found    |

---

## 15. Типові помилки

❌ Використання `request.POST`

❌ Відсутність Serializer

❌ Плутанина Django views і DRF views

❌ Відсутність статус-кодів

---

## 16. Коротке резюме

* REST = стандарт API
* DRF = надбудова над Django
* Serializer — ключовий елемент
* Response + request.data
* ViewSets економлять час

---

## Практичне завдання

1. Створіть модель Product
2. Реалізуйте API для list/create
3. Додайте serializer
4. Протестуйте через Browsable API
